In [1]:
import pandas as pd
import psycopg2 as pg2 

In [3]:
engine = pg2.connect("dbname='demo' user='postgres' host='localhost' port='5432' password='Qwerty000'")

In [16]:
query = """
WITH flight_passenger_count AS  (
	SELECT
		flight_id,
		CAST(count(boarding_no) AS numeric) AS passenger_count
	FROM
		boarding_passes
	GROUP BY
		flight_id
)

SELECT 
	flights.flight_id,
	flights.scheduled_departure,
	flights.departure_airport,
	flights.arrival_airport,
	--flights.status,
	flights.aircraft_code,
	aircrafts_data.model['en'],
	fpc.passenger_count,
	MAX(fpc.passenger_count) OVER(PARTITION BY flights.aircraft_code) AS max_capacity,
	CAST(fpc.passenger_count / MAX(fpc.passenger_count) OVER(PARTITION BY flights.aircraft_code) * 100 AS numeric(3)) AS capacity_perc,
	CAST(AVG(fpc.passenger_count) OVER(PARTITION BY flights.departure_airport, flights.arrival_airport) AS integer) AS avg_passenger_for_route,
	CAST(AVG(fpc.passenger_count) OVER(PARTITION BY flights.aircraft_code) AS integer) AS avg_passenger_for_aircraft,
	COUNT(flights.flight_id) OVER(PARTITION BY flights.departure_airport, flights.arrival_airport) AS count_flight_for_route,
	COUNT(flights.flight_id) OVER(PARTITION BY flights.aircraft_code) AS count_flight_for_aircraft,
	CASE 
		WHEN fpc.passenger_count = MAX(fpc.passenger_count) OVER(PARTITION BY flights.aircraft_code) THEN 'Заполнен'
		ELSE 'Не заполнен'
	END AS capacity_status
FROM 
	flights
	INNER JOIN flight_passenger_count AS fpc ON fpc.flight_id = flights.flight_id
	LEFT JOIN aircrafts_data ON aircrafts_data.aircraft_code = flights.aircraft_code
WHERE
	flights.status = 'Arrived'
	--AND flights.departure_airport = 'VVO'
	--AND flights.arrival_airport = 'KHV'
ORDER BY flights.scheduled_departure 
"""

In [20]:
df = pd.read_sql(query, engine)

C:\Users\Игорь\AppData\Local\Temp\ipykernel_24468\1589124803.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, engine)


In [21]:
df

,flight_id,scheduled_departure,departure_airport,arrival_airport,aircraft_code,model,passenger_count,max_capacity,capacity_perc,avg_passenger_for_route,avg_passenger_for_aircraft,count_flight_for_route,count_flight_for_aircraft,capacity_status
0,52986,2017-05-16 23:45:00+00:00,VVO,KHV,SU9,Sukhoi Superjet-100,39.0,97.0,40.0,79,62,90,10576,Не заполнен
1,31219,2017-05-17 00:20:00+00:00,YKS,LED,319,Airbus A319-100,5.0,116.0,4.0,13,60,91,1625,Не заполнен
2,52831,2017-05-17 00:55:00+00:00,VVO,IKT,CR2,Bombardier CRJ-200,1.0,50.0,2.0,8,24,89,11627,Не заполнен
3,49972,2017-05-17 01:35:00+00:00,IKT,LED,321,Airbus A321-200,12.0,170.0,7.0,132,95,91,1962,Не заполнен
4,52072,2017-05-17 02:05:00+00:00,KHV,LED,763,Boeing 767-300,24.0,219.0,11.0,118,125,91,1800,Не заполнен
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34381,40731,2017-08-15 13:35:00+00:00,GOJ,PEE,SU9,Sukhoi Superjet-100,34.0,97.0,35.0,32,62,90,10576,Не заполнен
34382,45589,2017-08-15 13:45:00+00:00,OVS,SGC,733,Boeing 737-300,38.0,130.0,29.0,36,88,90,1759,Не заполнен
34383,48268,2017-08-15 13:55:00+00:00,RTW,DME,CR2,Bombardier CRJ-200,27.0,50.0,54.0,24,24,91,11627,Не заполнен
34384,59330,2017-08-15 14:05:00+00:00,BZK,DME,SU9,Sukhoi Superjet-100,96.0,97.0,99.0,94,62,91,10576,Не заполнен


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34386 entries, 0 to 34385
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   flight_id                   34386 non-null  int64              
 1   scheduled_departure         34386 non-null  datetime64[ns, UTC]
 2   departure_airport           34386 non-null  object             
 3   arrival_airport             34386 non-null  object             
 4   aircraft_code               34386 non-null  object             
 5   model                       34386 non-null  object             
 6   passenger_count             34386 non-null  float64            
 7   max_capacity                34386 non-null  float64            
 8   capacity_perc               34386 non-null  float64            
 9   avg_passenger_for_route     34386 non-null  int64              
 10  avg_passenger_for_aircraft  34386 non-null  int64         

In [23]:
df.shape

(34386, 14)

In [28]:
print("Количество уникальных значений в колонке 'model': ", len(df['model'].unique()))

Количество уникальных значений в колонке 'model':  8


In [29]:
df['model'].value_counts()

model
Bombardier CRJ-200     11627
Sukhoi Superjet-100    10576
Cessna 208 Caravan      4130
Airbus A321-200         1962
Boeing 767-300          1800
Boeing 737-300          1759
Airbus A319-100         1625
Boeing 777-300           907
Name: count, dtype: int64

In [30]:
df['avg_passenger_for_route'].describe()

count    34386.000000
mean        54.946693
std         54.653369
min          2.000000
25%         17.000000
50%         41.000000
75%         78.000000
max        331.000000
Name: avg_passenger_for_route, dtype: float64